# Notebook testing out visualisation of the moon as a 3d sphere

Mostly taken from the tutorial here:
https://towardsdatascience.com/create-interactive-globe-earthquake-plot-in-python-b0b52b646f27

In [1]:
import plotly

In [6]:
import xarray as xa
import rioxarray as rio

In [24]:
data.x.size

4096

In [168]:
def get_data(fname = 'lroc_color_poles_4k.tif',
             ds_factor = 2):
    """ 
    Gets the data from a geotif with filename fname, downscaling int the processs
    Inputs:
        fname (str) : filename of geotif to load
        ds_factor (int) downscale factor
    Output:
        lon, lat, data ([n_lat x n_lon] np arrays)
    """
    import rioxarray as rio
    import xarray as xa
    import numpy as np
    from rasterio.enums import Resampling
    
    xds = rio.open_rasterio(fname)
    xds.attrs['crs'] = 'EPSG:4326'
    downscale_factor = 2
    new_width = xds.rio.width // ds_factor
    new_height = xds.rio.height // ds_factor
    xds.var

    xds_resampled = xds.rio.reproject(
        xds.rio.crs,
        shape=(new_height, new_width),
        resampling=Resampling.bilinear,
        SRC_METHOD='NO_GEOTRANSFORM'
    )
    lon = xds_resampled.x.data
    lat = xds_resampled.y.data
    
    lon = (lon - lon.min()) / (lon.max() - lon.min()) * 360 - 180 
    lat = (lat - lat.min()) / (lat.max() - lat.min()) * 180 - 90 


    
    
    lon, lat = np.meshgrid(lon, lat)
    
    return lon, lat, xds_resampled.data[0, ...]

In [171]:
import numpy as np
def degree2radians(degree):
  # convert degrees to radians
  return degree*np.pi/180
  
def mapping_map_to_sphere(lon, lat, radius=1):
    # this function maps the points of coords (lon, lat) to points onto the sphere of radius radius
    lon=np.array(lon, dtype=np.float64)
    lat=np.array(lat, dtype=np.float64)
    lon=degree2radians(lon)
    lat=degree2radians(lat)
    xs=radius*np.cos(lon)*np.cos(lat)
    ys=radius*np.sin(lon)*np.cos(lat)
    zs=radius*np.sin(lat)
    return xs, ys, zs


In [180]:
# Get mesh-shape topography data
lon, lat, greys = get_data(ds_factor = 2)

In [181]:


# Locations on the sphere fro grid cells in x,y,z coords
xs, ys, zs = mapping_map_to_sphere(lon, lat)

In [182]:
np.max(lon)

180.0

In [186]:
Ctopo = [[0, 'rgb(0, 0, 0)'],[1.0, 'rgb(255,255,255)']]

In [187]:
cmin = 0
cmax = 210
topo_sphere=dict(type='surface',
  x=xs,
  y=ys,
  z=zs,
  colorscale='greys',
  surfacecolor=greys,
  cmin=cmin,
  cmax=cmax)

In [188]:
import plotly.graph_objs as go

titlecolor = 'white'
bgcolor = 'black'




noaxis=dict(showbackground=False,
  showgrid=False,
  showline=False,
  showticklabels=False,
  ticks='',
  title='',
  zeroline=False)

layout = go.Layout(
  autosize=False, width=1200, height=800,
  title = '3D spherical topography map',
  titlefont = dict(family='Courier New', color=titlecolor),
  showlegend = False,
  scene = dict(
    xaxis = noaxis,
    yaxis = noaxis,
    zaxis = noaxis,
    aspectmode='manual',
    aspectratio=go.layout.scene.Aspectratio(
      x=1, y=1, z=1)),
  paper_bgcolor = bgcolor,
  plot_bgcolor = bgcolor)

In [189]:
from plotly.offline import plot

plot_data=[topo_sphere]
fig = go.Figure(data=plot_data, layout=layout)
plot(fig, validate = False, filename='SphericalTopography.html',
   auto_open=True)

'SphericalTopography.html'